In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
%cd "/content/drive/My Drive/Research/FIRE/2020-Speech-Recognition/"

/content/drive/My Drive/Research/FIRE/2020-Speech-Recognition


In [3]:
!ls

amazing_grace	       Data_Visualization.ipynb  Model
Baseline_Model.ipynb   environment_checker.py	 __pycache__
benchmark_speed.ipynb  environment.yml		 README.md
DataAugmentation.py    generator_test.ipynb	 Test_Data_Augmentation.ipynb
data_checker.py        input_pipeline.ipynb
DataGenerator.py       keys.txt


In [4]:
!pip install python_speech_features
!pip install tensorflow==2.1.0

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp36-none-any.whl size=5887 sha256=f519fff97f07517b36b4179a7e788abe941948202c5b0640a17963f1d4f67f9d
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features
     |████████████████████████████████| 421.8MB 35kB/s 
     |████████████████████████████████| 450kB 46.6MB/s 
     |████████████████████████████████| 3.9MB 43.8MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=5bc6e826364231b217b18c7a210c9f4f23401c0474d6864879404bb07ae63569
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalli

In [5]:
import sys
sys.path.append("Model")
from alphabet import Alphabet
alphabet = Alphabet("Model/alphabet.txt")

In [6]:
from model_utils import get_compiled_model
from model_utils import get_decoder
from model_utils import get_callbacks

compiled_model = get_compiled_model()
callbacks = get_callbacks()
decoder = get_decoder(compiled_model.output, alphabet)

calculating loss
Tensor("loss/time_distributed_2_loss/ExpandDims:0", shape=(None, 1), dtype=float32)
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Tensor("SparseToDense:0", shape=(None, None), dtype=int64)


In [36]:
compiled_model.load_weights("Model/checkpoints/weights.05-131.86.hdf5")

In [25]:
import numpy as np
import python_speech_features
import librosa

conf = {"winfunc": np.hamming, "winlen": 0.025, "winstep": .01, "nfilt": 80}

def get_features(files):
    mfccs = [make_features(file) for file in files]
    X = align(mfccs)
    return X

def make_features(file_path):
    """ Use `python_speech_"features` lib to extract MFCC features from the audio file. """
    audio, fs = librosa.load(file_path, sr=16000)
    audio = (audio * 32768).astype("int16")
    feat, energy = python_speech_features.fbank(audio, samplerate=fs, **conf)
    features = np.log(feat)
    return features

def align(arrays, default=0):
    """ Pad arrays along time dimensions. Return the single array (batch_size, time, features). """
    max_array = max(arrays, key=len)
    X = np.full(shape=[len(arrays), *max_array.shape], fill_value=default, dtype=np.float64)
    for index, array in enumerate(arrays):
        time_dim, features_dim = array.shape
        X[index, :time_dim] = array
    return X

In [14]:
files = ["/content/saturday_nights.mp3"]
audio, fs = librosa.load(files[0], sr=16000)

from IPython.display import Audio
Audio(audio, rate=fs)

Output hidden; open in https://colab.research.google.com to view.

In [37]:
X = get_features(files)
y = compiled_model.predict_on_batch(X)
decoder(y)

[' ']

In [40]:
y.argmax(axis=0)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [41]:
%cd /content

/content


In [47]:
from google.cloud import storage
bucket_name = 'automatic-speech-recognition'
lang = 'en'
name = 'deepspeech'
version = '1.0'
remote_path = f'{lang}-{name}-weights-{version}.h5'
local_path = "./weights.h5"

client = storage.Client.create_anonymous_client()
bucket = client.bucket(bucket_name)
blob = storage.Blob(remote_path, bucket)
blob.download_to_filename(local_path, client=client)

NotFound: ignored